In [5]:
# boxing_dashboard.py  (multi-bookmaker charts, fixed event_id in best table)
import sqlite3
import pandas as pd
import json

DB_PATH = "boxing_odds_staging.db"
OUTPUT_HTML = "boxing_dashboard.html"

# Optional: bookmaker logos
BOOKMAKER_LOGOS = {
    # "Virgin Bet": "https://.../virginbet.png",
    # "Betfair": "https://.../betfair.png",
    # "Unibet": "https://.../unibet.png",
    # "Grosvenor": "https://.../grosvenor.png",
    # "LiveScore Bet": "https://.../livescorebet.png",
}

# -----------------------------
# 1) Load data from SQLite
# -----------------------------
conn = sqlite3.connect(DB_PATH)

latest_sql = """
SELECT *
FROM odds_history
WHERE (event_id, fighter, bookmaker, observed_at) IN (
    SELECT event_id, fighter, bookmaker, MAX(observed_at)
    FROM odds_history
    GROUP BY event_id, fighter, bookmaker
)
"""
latest = pd.read_sql(latest_sql, conn)

history = pd.read_sql("""
SELECT event_id, fighter, bookmaker, observed_at, decimal_odds, commence_time, home_team, away_team
FROM odds_history
""", conn)

conn.close()

# Normalize/parse times
latest["observed_at"]   = pd.to_datetime(latest["observed_at"],   errors="coerce", utc=True)
latest["commence_time"] = pd.to_datetime(latest["commence_time"], errors="coerce", utc=True)
history["observed_at"]  = pd.to_datetime(history["observed_at"],  errors="coerce", utc=True)
history["commence_time"]= pd.to_datetime(history["commence_time"],errors="coerce", utc=True)

# Build matchup + fight date
latest["Matchup"]    = latest["home_team"].fillna("") + " vs " + latest["away_team"].fillna("")
latest["Fight Date"] = latest["commence_time"].dt.date

# Remove Draw rows for Best Odds section
latest_no_draw = latest[latest["fighter"] != "Draw"].copy()

# Best Odds per fighter (row with MAX decimal_odds) — keep needed columns incl. event_id
idx = latest_no_draw.groupby(["event_id", "fighter"])["decimal_odds"].idxmax()
best = latest_no_draw.loc[idx, [
    "event_id", "fighter", "bookmaker", "decimal_odds", "Matchup", "Fight Date"
]].copy()
best["Implied %"] = (100.0 / best["decimal_odds"]).round(2)
best.sort_values(["Fight Date", "Matchup", "decimal_odds"], ascending=[True, True, False], inplace=True)

# -----------------------------
# 2) Multi-bookmaker chart data per (event_id, fighter)
# -----------------------------
# Dict: key -> { label, series: [ {name, times[], odds[]} ] }
hist_for_js = {}
for (ev, fighter), grp in history[history["fighter"] != "Draw"].groupby(["event_id", "fighter"]):
    series = []
    for bk, bk_df in grp.sort_values("observed_at").groupby("bookmaker"):
        series.append({
            "name": bk,
            "times": bk_df["observed_at"].dt.strftime("%Y-%m-%d %H:%M").tolist(),
            "odds":  bk_df["decimal_odds"].round(3).tolist()
        })
    key = f"{ev}__{fighter}".replace(" ", "_")
    hist_for_js[key] = {
        "label": f"{fighter} — all bookmakers",
        "series": series
    }

# -----------------------------
# 3) Biggest Movers (last 24h)
# -----------------------------
if not history.empty:
    now = history["observed_at"].max()
else:
    now = pd.Timestamp.utcnow()

window_start = now - pd.Timedelta(hours=24)
mov = history[(history["observed_at"] >= window_start) & (history["observed_at"] <= now)].copy()

if not mov.empty:
    first_idx = mov.sort_values("observed_at").groupby(["event_id", "fighter"])["observed_at"].idxmin()
    last_idx  = mov.sort_values("observed_at").groupby(["event_id", "fighter"])["observed_at"].idxmax()
    first = mov.loc[first_idx, ["event_id", "fighter", "decimal_odds"]].rename(columns={"decimal_odds": "odds_first"})
    last_  = mov.loc[last_idx,  ["event_id", "fighter", "decimal_odds"]].rename(columns={"decimal_odds": "odds_last"})
    movers = pd.merge(first, last_, on=["event_id", "fighter"], how="inner")
    movers["abs_change"] = (movers["odds_last"] - movers["odds_first"]).round(3)
    movers["pct_change"] = ((movers["odds_last"] - movers["odds_first"]) / movers["odds_first"] * 100).round(2)

    attach_cols = latest[["event_id", "home_team", "away_team", "commence_time"]].drop_duplicates("event_id")
    movers = movers.merge(attach_cols, on="event_id", how="left")
    movers["Matchup"] = movers["home_team"].fillna("") + " vs " + movers["away_team"].fillna("")
    movers["Fight Date"] = movers["commence_time"].dt.date

    movers["imp_first_%"] = (100.0 / movers["odds_first"]).round(2)
    movers["imp_last_%"]  = (100.0 / movers["odds_last"]).round(2)
    movers["imp_delta_%"] = (movers["imp_last_%"] - movers["imp_first_%"]).round(2)
    movers = movers.sort_values("pct_change", key=lambda s: s.abs(), ascending=False).head(12)
else:
    movers = pd.DataFrame(columns=[
        "event_id","fighter","odds_first","odds_last","abs_change","pct_change",
        "Matchup","Fight Date","imp_first_%","imp_last_%","imp_delta_%"
    ])

# -----------------------------
# 4) Render HTML helpers
# -----------------------------
def bookmaker_cell(name: str) -> str:
    if name in BOOKMAKER_LOGOS and BOOKMAKER_LOGOS[name]:
        return f"<div class='bk-cell'><img src='{BOOKMAKER_LOGOS[name]}' alt='{name}'/> <span>{name}</span></div>"
    return f"<div class='bk-cell'><span>{name}</span></div>"

def best_table_html(df: pd.DataFrame) -> str:
    parts = []
    for d, grp in df.groupby("Fight Date"):
        parts.append(f"<div class='card'><div class='card-title'>Fights on {d}</div>")
        parts.append("""
        <table class="odds">
          <thead>
            <tr>
              <th>Matchup</th>
              <th>Fighter</th>
              <th>Best Bookmaker</th>
              <th>Best Odds</th>
              <th>Implied %</th>
              <th>Chart</th>
            </tr>
          </thead>
          <tbody>
        """)
        for _, r in grp.iterrows():
            chart_key = f'{r["event_id"]}__{r["fighter"]}'.replace(" ", "_")
            parts.append(f"""
              <tr class="clickable" onclick="toggleChart('{chart_key}', 'icon_{chart_key}')">
                <td>{r["Matchup"]}</td>
                <td>{r["fighter"]} <span id="icon_{chart_key}" class="toggle">▼</span></td>
                <td>{bookmaker_cell(r["bookmaker"])}</td>
                <td><b>{round(r["decimal_odds"],3)}</b></td>
                <td>{r["Implied %"]}%</td>
                <td><button class="btn" type="button">View</button></td>
              </tr>
              <tr id="row_{chart_key}" class="chart-row">
                <td colspan="6">
                  <div id="chart_{chart_key}" class="chart"></div>
                </td>
              </tr>
            """)
        parts.append("</tbody></table></div>")
    return "\n".join(parts)

def movers_table_html(df: pd.DataFrame) -> str:
    if df.empty:
        return "<div class='card'><div class='card-title'>Biggest Movers (24h)</div><div class='empty'>No data in last 24h.</div></div>"
    rows = []
    for _, r in df.iterrows():
        cls = "pos" if r["pct_change"] > 0 else "neg" if r["pct_change"] < 0 else ""
        rows.append(f"""
          <tr>
             <td>{r.get("Fight Date","")}</td>
             <td>{r.get("Matchup","")}</td>
             <td>{r["fighter"]}</td>
             <td>{round(r["odds_first"],3)} → <b>{round(r["odds_last"],3)}</b></td>
             <td class="{cls}">{r["pct_change"]}%</td>
             <td>{r["imp_first_%"]}% → <b>{r["imp_last_%"]}%</b> ({r["imp_delta_%"]}% Δ)</td>
          </tr>
        """)
    return f"""
    <div class='card'>
      <div class='card-title'>Biggest Movers (last 24h)</div>
      <table class="odds">
        <thead>
          <tr>
            <th>Fight Date</th>
            <th>Matchup</th>
            <th>Fighter</th>
            <th>Odds (first → last)</th>
            <th>% Change</th>
            <th>Implied % (first → last)</th>
          </tr>
        </thead>
        <tbody>
          {''.join(rows)}
        </tbody>
      </table>
    </div>
    """

best_html   = best_table_html(best)
movers_html = movers_table_html(movers)
charts_json = json.dumps(hist_for_js)

# -----------------------------
# 5) Full HTML (dark UI)
# -----------------------------
html = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<title>Boxing Odds Dashboard</title>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<style>
  :root {{
    --bg: #0c1220;
    --card: #111a2e;
    --muted: #7b87a1;
    --text: #e8ecf3;
    --accent: #00d4ff;
    --green: #16c784;
    --red: #ea3943;
    --border: #24304a;
  }}
  * {{ box-sizing: border-box; }}
  body {{
    margin: 0; padding: 24px;
    font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Inter, Arial;
    background: linear-gradient(180deg, #0c1220 0%, #0c1425 100%);
    color: var(--text);
  }}
  .container {{ max-width: 1200px; margin: 0 auto; }}
  header {{
    display:flex; align-items:center; justify-content:space-between;
    margin-bottom: 24px;
  }}
  header .title {{
    font-size: 22px; font-weight: 700; letter-spacing: .2px;
  }}
  .pill {{ color: #09121f; background: var(--accent); padding: 6px 10px; border-radius: 999px; font-weight: 700; }}
  .grid {{
    display:grid; gap: 16px;
    grid-template-columns: 1fr;
  }}
  @media (min-width: 900px) {{
    .grid {{ grid-template-columns: 2fr 1fr; }}
  }}
  .card {{
    background: var(--card);
    border: 1px solid var(--border);
    border-radius: 16px;
    padding: 16px;
    box-shadow: 0 8px 30px rgba(0,0,0,.25);
  }}
  .card-title {{ font-weight:700; margin-bottom:12px; }}
  .odds {{
    width:100%; border-collapse: collapse; overflow:hidden; border-radius: 12px;
  }}
  .odds th, .odds td {{
    padding: 12px 14px; border-bottom: 1px solid var(--border); text-align: left;
  }}
  .odds thead th {{ color: #b8c3de; background: #0f1a2f; position: sticky; top:0; z-index:1; }}
  .odds tr.clickable:hover {{ background: #0f223f; cursor: pointer; }}
  .toggle {{ float:right; color: var(--muted); font-size: 12px; }}
  .btn {{
    background: #112a46; color: #bfe9ff; border: 1px solid #1c3d63;
    padding: 6px 10px; border-radius: 8px; font-weight:600;
  }}
  .chart-row {{ display: none; }}
  .chart {{ width:100%; height:360px; background:#0b111f; border:1px solid var(--border); border-radius: 12px; }}
  .empty {{ color: var(--muted); padding: 12px 0; }}
  .pos {{ color: var(--green); font-weight:700; }}
  .neg {{ color: var(--red); font-weight:700; }}
  .bk-cell {{ display:flex; align-items:center; gap:8px; }}
  .bk-cell img {{ height:18px; border-radius:4px; }}
</style>
</head>
<body>
<div class="container">
  <header>
    <div class="title">Boxing Odds Dashboard</div>
    <div class="pill">Beta</div>
  </header>

  <div class="grid">
    <div>
      {best_html}
    </div>
    <div>
      {movers_html}
    </div>
  </div>
</div>

<script>
  // Pre-baked chart data from Python
  const CHARTS = {charts_json};

  // Toggle + lazy render a fighter's chart (multi-bookmaker)
  function toggleChart(key, iconId) {{
    const row = document.getElementById('row_' + key);
    const icon = document.getElementById(iconId);
    const div  = document.getElementById('chart_' + key);

    if (row.style.display === 'table-row') {{
      row.style.display = 'none';
      if (icon) icon.textContent = '▼';
      return;
    }}
    row.style.display = 'table-row';
    if (icon) icon.textContent = '▲';

    if (div.getAttribute('data-rendered') === '1') return;

    const payload = CHARTS[key];
    if (!payload || !payload.series || payload.series.length === 0) {{
      div.innerHTML = "<div style='padding:12px;color:#89a;'>No history available.</div>";
      div.setAttribute('data-rendered','1');
      return;
    }}

    const traces = payload.series.map(s => ({
      x: s.times,
      y: s.odds,
      mode: 'lines+markers',
      type: 'scatter',
      name: s.name
    }));

    const layout = {{
      paper_bgcolor: '#0b111f',
      plot_bgcolor: '#0b111f',
      font: {{ color: '#d9e2f1' }},
      margin: {{ t:40, r:12, b:40, l:50 }},
      title: payload.label,
      xaxis: {{ title: 'Time', gridcolor: '#1d2a45' }},
      yaxis: {{ title: 'Decimal Odds', gridcolor: '#1d2a45' }},
      legend: {{ orientation: 'h', y: -0.2 }}
    }};

    Plotly.newPlot(div, traces, layout, {{displayModeBar:false, responsive:true}});
    div.setAttribute('data-rendered','1');
  }}
</script>
</body>
</html>
"""

with open(OUTPUT_HTML, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ Dashboard written to { OUTPUT_HTML }")


NameError: name 'x' is not defined